# Interpretabilidad Mecanicista de Transformers

## Introducción

Este documento detalla el proceso para descargar e implementar el modelo **Llama 3.2 1B en fp16** desde **Hugging Face**, extrayendo la *n*-ésima salida de la capa MLP (*Multi-Layer Perceptron*). También se documentará cada paso siguiendo una metodología.

## ¿Qué es Llama 3.2 1B?

Llama 3.2 es un modelo de lenguaje basado en la arquitectura **Transformer**, desarrollado por Meta. Su tamaño (1B de parámetros) lo hace eficiente para tareas de procesamiento del lenguaje natural.

## ¿Qué es fp16 y por qué es importante?

**fp16 (Floating Point 16 bits)** es un formato de precisión reducida que permite acelerar el entrenamiento y la inferencia del modelo, reduciendo el uso de memoria sin perder mucha precisión.

## ¿Qué es Hugging Face?

**Hugging Face** es una plataforma líder en el desarrollo de modelos de inteligencia artificial, especialmente en el campo del procesamiento del lenguaje natural (NLP). Proporciona una gran variedad de modelos preentrenados, herramientas para el entrenamiento y despliegue de modelos, y una comunidad activa de investigadores y desarrolladores.

### ¿Para qué se usa Hugging Face?

- **Repositorio de modelos:** Permite descargar y compartir modelos preentrenados.
- **Transformers Library:** Proporciona una API para usar modelos de NLP fácilmente.
- **Hugging Face Hub:** Un espacio para colaborar y almacenar modelos y datasets.
- **Inference API:** Para probar modelos sin necesidad de descargarlos localmente.

### Conceptos clave que usaremos en Hugging Face

1. **Token de autenticación:** Necesario para acceder a modelos privados o restringidos.
2. **Modelos preentrenados:** Conjuntos de pesos y configuraciones listos para usar.
3. **Tokenizer:** Convierte texto en tensores numéricos que el modelo puede procesar.
4. **Pipeline:** Interfaz sencilla para ejecutar modelos en tareas específicas.
5. **AutoModel y AutoTokenizer:** Clases que nos permiten cargar modelos y tokenizadores sin necesidad de conocer su arquitectura exacta.

---

## Acceso al modelo en Hugging Face

Para acceder a *Llama 3.2*, hay dos métodos principales:

### Método 1: Solicitud de acceso manual 

1. Ir a [Hugging Face](https://huggingface.co/).
2. Usar la barra de búsqueda para encontrar **Llama 3.2 1B**.
3. Llenar el formulario de solicitud de acceso.
4. Una vez aprobado, podrás acceder a los archivos del modelo en la pestaña **Files and versions**.

Este método es útil si el modelo tiene restricciones de acceso y no requiere autenticación en código.

### Método 2: Autenticación con un token de acceso

Si necesitas automatizar el proceso o descargar modelos privados, puedes generar un token de acceso:

1. Ve a **Settings > Access Tokens** en Hugging Face.
2. Genera un nuevo *token* con permisos de "read".
3. Usa el siguiente código para autenticarte:

```python
from huggingface_hub import login
login("TU_TOKEN_AQUI")  # Reemplaza con tu token
```

Este método es más formal y útil si trabajas en servidores o con varios modelos en diferentes proyectos.

---

## Configuración del entorno

Instalaremos las dependencias necesarias en nuestro entorno de Python:

```bash
pip install torch torchvision torchaudio transformers huggingface_hub
```

Verificamos la instalación ejecutando:

```bash
python3 -c "import torch; print(torch.__version__)"
```

Si el comando muestra un número de versión (`2.x.x`), significa que **PyTorch está correctamente instalado**.

Opcionalmente, podemos crear un entorno virtual (quizá esrte paso lo debas hacer primero para poder instalar paquetes, ya deoendera de tu gestor de paquetes que tengas en uso):

```bash
python -m venv llama_env
source llama_env/bin/activate  # En macOS/Linux
llama_env\Scripts\activate  # En Windows
```

---

## Descarga del modelo desde Hugging Face

Cargaremos el modelo y el *tokenizer* desde Hugging Face:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B"  # Nombre exacto del modelo

# Cargar el tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el tokenizer: {e}")

# Cargar el modelo completo
try:
    model = AutoModelForCausalLM.from_pretrained(model_name)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
```

Si el modelo se descarga correctamente, estará listo para su análisis y uso en inferencias. En caso de errores, verifica los permisos de acceso en Hugging Face o la correcta instalación de las dependencias.

---

## Estructura del modelo Llama 3.2 1B

Llama 3.2 1B sigue la arquitectura **Transformer**, que está compuesta por múltiples bloques de procesamiento de información. Sus principales componentes son:

###  1. **Embeddings**
Los embeddings convierten palabras o tokens en vectores numéricos de alta dimensión. Estos vectores son la entrada del modelo y representan el significado de las palabras en un espacio matemático.

###  2. **Múltiples capas de atención (Self-Attention)**
Cada capa de atención analiza las relaciones entre todas las palabras de la oración para determinar cuáles son más relevantes para la predicción.

###  3. **Capa MLP (Multi-Layer Perceptron)**
Después de cada capa de atención, los datos pasan por una **MLP (Red Neuronal de Múltiples Capas)**. Esta capa:
   - Refina la información extraída por la atención.
   - Introduce no linealidad al modelo.
   - Generaliza mejor las representaciones del lenguaje.

Cada MLP en el Transformer tiene dos capas completamente conectadas con una función de activación no lineal intermedia.

###  4. **Capa de salida**
Finalmente, la capa de salida del modelo convierte la representación final en una probabilidad de predicción sobre el siguiente token en la secuencia.

###  5. **Estructura en profundidad**
Llama 3.2 1B tiene varias capas Transformer apiladas, donde cada capa tiene una subcapa de **self-attention** y una subcapa MLP. Las activaciones intermedias dentro de la MLP son esenciales para analizar la información que el modelo está aprendiendo en cada paso.

---

## Guardar y cargar el modelo localmente

Para evitar descargar el modelo cada vez que lo necesitemos, podemos almacenarlo localmente:

```python
# Guardar el modelo localmente
model.save_pretrained("./llama3_model")
tokenizer.save_pretrained("./llama3_model")

# Cargarlo más tarde sin conexión
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./llama3_model")
model = AutoModelForCausalLM.from_pretrained("./llama3_model")
```

Esto optimiza el tiempo y evita depender de internet en cada ejecución.

---

## Generar texto con Llama 3.2

Una vez que el modelo está cargado, podemos probarlo generando texto:

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Definir el modelo
model_name = "meta-llama/Llama-3.2-1B"

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Verificar si MPS está disponible y mover el modelo a GPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Tokenizar la entrada y mover a GPU
prompt = "Explica la importancia de la interpretabilidad en transformers."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generar texto con MPS activado
print("Generando texto...")
with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_length=100,  # Reducido para mejorar velocidad
        temperature=0.7,  # Controla la aleatoriedad
        top_p=0.9  # Controla la diversidad de respuestas
    )
print("Generación completada.")

# Decodificar la salida del modelo
texto_generado = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Texto generado:", texto_generado)

```

Este código permite ver cómo Llama 3.2 responde a una consulta en lenguaje natural.


# Explorando la arquitectura interna de Llama 3.2 1B

Antes de extraer salidas intermedias, es fundamental comprender cómo está estructurado el modelo.

## 1️ Ver la configuración general del modelo

La configuración del modelo contiene información clave como:
- Número de capas (`num_hidden_layers`)
- Dimensión de los embeddings (`hidden_size`)
- Número de cabezas de atención (`num_attention_heads`)
- Tamaño de la capa intermedia MLP (`intermediate_size`)

```python
from transformers import AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ver configuración general del modelo
print(model.config)
```

### Explicación detallada de la configuración del modelo:

- **_name_or_path**: indica el nombre exacto del modelo que estamos utilizando.
- **architectures**: muestra la clase principal usada para la arquitectura, en este caso `LlamaForCausalLM`.
- **attention_bias**: especifica si hay un sesgo aplicado en las matrices de atención (aquí es `false`).
- **attention_dropout**: nivel de abandono (dropout) aplicado en la atención (0.0 significa sin dropout).
- **bos_token_id / eos_token_id**: identificadores especiales de inicio y fin de secuencia.
- **head_dim**: tamaño de la dimensión de cada cabeza de atención.
- **hidden_act**: función de activación en la MLP (aquí `silu`).
- **hidden_size**: tamaño de la representación oculta, es decir, la dimensión del embedding (2048).
- **initializer_range**: rango usado para inicializar los pesos.
- **intermediate_size**: tamaño de la capa intermedia de la MLP (8192).
- **max_position_embeddings**: el máximo número de posiciones que puede procesar el modelo (131072).
- **mlp_bias**: indica si la MLP usa sesgos (falso en este caso).
- **model_type**: tipo de modelo (`llama`).
- **num_attention_heads**: número de cabezas de atención (32).
- **num_hidden_layers**: número de capas Transformer (16).
- **num_key_value_heads**: número de cabezas para valores y llaves (8).
- **pretraining_tp**: indica partición para entrenamiento (1 significa sin particiones).
- **rms_norm_eps**: valor epsilon para estabilidad numérica en normalización.
- **rope_scaling**: contiene parámetros relacionados con el mecanismo de rotación posicional (RoPE).
- **rope_theta**: parámetro adicional para la frecuencia rotacional (500000.0).
- **tie_word_embeddings**: indica si las embeddings de entrada y salida están ligadas.
- **torch_dtype**: tipo de datos usado (`float32`).
- **transformers_version**: versión de la librería Transformers usada (4.49.0).
- **use_cache**: indica si se utiliza cache para acelerar inferencias.
- **vocab_size**: tamaño del vocabulario (128256 tokens).

---

## 2️ Ver cuántos bloques Transformer tiene el modelo

El modelo tiene una lista de capas accesible con `model.model.layers`. Cada elemento es un bloque Transformer completo.

```python
# Visualizar la lista de bloques Transformer
print(model.model.layers)
```

### Explicación línea por línea del resultado:

```plaintext
ModuleList(
  (0-15): 16 x LlamaDecoderLayer(
```
- `ModuleList`: indica que es una lista de módulos (capas).
- `(0-15): 16 x LlamaDecoderLayer`: el modelo tiene 16 capas numeradas de la 0 a la 15, cada una es un `LlamaDecoderLayer`.

```plaintext
    (self_attn): LlamaAttention(
```
- Cada capa contiene un módulo de autoatención (`self_attn`).

```plaintext
      (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
```
- `q_proj`: capa lineal que proyecta la consulta (query) desde 2048 a 2048 dimensiones.

```plaintext
      (k_proj): Linear(in_features=2048, out_features=512, bias=False)
```
- `k_proj`: proyección de la clave (key) de 2048 dimensiones a 512.

```plaintext
      (v_proj): Linear(in_features=2048, out_features=512, bias=False)
```
- `v_proj`: proyección del valor (value) de 2048 a 512 dimensiones.

```plaintext
      (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
```
- `o_proj`: proyecta la salida de vuelta a 2048 dimensiones.

```plaintext
    )
```
- Fin del módulo de atención.

```plaintext
    (mlp): LlamaMLP(
```
- Comienza la descripción del módulo MLP (multi-layer perceptron).

```plaintext
      (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
```
- `gate_proj`: primera capa lineal que expande la dimensión de 2048 a 8192.

```plaintext
      (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
```
- `up_proj`: otra proyección de expansión.

```plaintext
      (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
```
- `down_proj`: reduce nuevamente de 8192 dimensiones a 2048.

```plaintext
      (act_fn): SiLU()
```
- `act_fn`: la función de activación usada es SiLU (Sigmoid Linear Unit).

```plaintext
    )
```
- Fin del módulo MLP.

```plaintext
    (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
```
- `input_layernorm`: normalización de entrada.

```plaintext
    (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
```
- `post_attention_layernorm`: normalización después del bloque de atención.

```plaintext
  )
)
```

---


## 3️ Analizar un bloque Transformer específico

Para entender mejor la estructura interna, accedemos al primer bloque (índice 0):

```python
# Acceder al primer bloque Transformer
primer_bloque = model.model.layers[0]

# Ver la estructura interna del primer bloque
print(primer_bloque)
```
Al imprimir el contenido de un bloque Transformer, observamos la estructura de un **LlamaDecoderLayer**, que es la unidad básica repetida en el modelo.

# Estructura del `LlamaDecoderLayer`

```python
LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)
```

# Desglose de cada componente:

#  **(self_attn): LlamaAttention**
Este bloque es el mecanismo de atención, compuesto por:
- **q_proj**: Proyección lineal de las queries.
- **k_proj**: Proyección lineal de las keys (notar que reduce de 2048 a 512 dimensiones).
- **v_proj**: Proyección lineal de los values (también reduce de 2048 a 512 dimensiones).
- **o_proj**: Proyección lineal para combinar el resultado de la atención (vuelve a 2048).

###  **(mlp): LlamaMLP**
Es la red neuronal de múltiples capas, compuesta por:
- **gate_proj**: Proyección lineal que lleva de 2048 a 8192 dimensiones.
- **up_proj**: Otra proyección lineal de 2048 a 8192.
- **down_proj**: Reduce de 8192 de vuelta a 2048 dimensiones.
- **act_fn**: La función de activación no lineal **SiLU()**.

###  **Normalizaciones**
- **input_layernorm**: Normalización RMS antes de la atención, estabiliza la entrada.
- **post_attention_layernorm**: Otra normalización RMS aplicada después del bloque de atención.

## Observaciones importantes:
- Las proyecciones lineales con `bias=False` indican que no hay término constante agregado.
- La reducción de dimensionalidad en las proyecciones de keys y values ayuda a ahorrar memoria y computación.
- La MLP amplifica la representación (multiplicando por 4 el tamaño del vector) y luego la comprime, lo cual ayuda a capturar relaciones complejas.


---

## 4️ La MLP dentro de un bloque Transformer

Cada bloque Transformer contiene un submódulo `mlp` que es un perceptrón multicapa con dos capas lineales y una activación no lineal.

```python
# Visualizar la estructura de la MLP dentro del primer bloque
print(primer_bloque.mlp)
```

Al imprimir el contenido de la MLP dentro de un bloque Transformer, vemos la estructura siguiente:

```python
LlamaMLP(
  (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
  (act_fn): SiLU()
)
```

# Descripción de cada componente

  **gate_proj**
- Es una capa lineal que transforma un vector de dimensión 2048 a 8192.
- El nombre *gate* indica que se utiliza junto con una función de activación para controlar qué información pasa y qué se bloquea.
- No tiene sesgo (`bias=False`), lo que significa que solo es una multiplicación lineal.

  **up_proj**
- También proyecta de 2048 a 8192 dimensiones.
- Funciona en conjunto con `gate_proj` para ampliar la representación interna.

 **act_fn: SiLU()**
- La función de activación es **SiLU** (*Sigmoid Linear Unit*), que introduce no linealidad en la transformación.
- Esta función es suave y se ha demostrado eficaz en modelos grandes.

 **down_proj**
- Una vez ampliada y transformada la representación, esta capa la reduce de nuevo de 8192 a 2048 dimensiones.
- La compresión permite que la información relevante pase, eliminando redundancia y permitiendo un procesamiento eficiente.

 ¿Por qué estas proyecciones?  
La MLP dentro de un Transformer actúa como una red que:
1. Expande la representación (de 2048 a 8192).
2. Aplica una activación no lineal.
3. Comprime nuevamente a 2048 dimensiones.

Este proceso permite que la red capture relaciones más complejas y refinadas que no podrían obtenerse solo mediante capas de atención.




## Interceptando la salida de la MLP en la capa n-ésima

Para entender qué información está procesando el modelo, vamos a interceptar la salida de la MLP en una capa específica.

### ¿Cómo hacerlo?

Utilizaremos *hooks* de PyTorch.  
Un *hook* es una función que se ejecuta automáticamente cada vez que pasa información por una capa específica.  
Así podremos capturar la salida intermedia sin modificar el modelo.

---

### Paso 1️: Definir una función hook

Un hook es una función que se ejecuta cada vez que la capa procesa información.
Esto nos permitirá interceptar y guardar la salida intermedia de la MLP.

```python
# Diccionario para guardar las activaciones
activaciones_mlp = {}

# Función hook para almacenar la salida
def guardar_salida_mlp(layer_num):
    def hook(module, input, output):
        activaciones_mlp[f'capa_{layer_num}'] = output.detach().cpu()
    return hook
```
**Explicación:**
- `activaciones_mlp` es un diccionario donde almacenamos la salida de la MLP.
- `guardar_salida_mlp(layer_num)` devuelve una función interna (hook) que captura el `output` de la MLP cuando se ejecuta el `forward`.
- `output.detach().cpu()` desconecta el tensor del grafo de cómputo y lo pasa a la CPU para ahorrar memoria.

---

### Paso 2️: Registrar el hook en la capa deseada

Elegimos el número de capa `n` en la cual queremos interceptar la salida y registramos el hook.

```python
n = 5  # este numero lo cambiamos a gusto para interceptar la capa que se desee
handle = model.model.layers[n].mlp.register_forward_hook(guardar_salida_mlp(n))
```
**Explicación:**
- Se elige la capa `n` donde quieres capturar la salida.
- `register_forward_hook()` vincula la función hook a la capa MLP de ese bloque.
---

### Paso 3️: Ejecutar una inferencia para activar el hook

Realizamos una inferencia normal; el hook capturará la salida automáticamente.
# si no tienes definido donde correras el modelo, definirlo ahora con torch.device("mps" if torch.backends.mps.is_available() else 2cpu")
```python
prompt = "La interpretabilidad en transformers es fundamental."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    _ = model(**inputs)
```
**Explicación:**
- Definimos el dispositivo para que el modelo y las entradas trabajen en la misma plataforma.
- El modelo ejecuta un `forward` sobre el `prompt`, y gracias al hook, se almacena automáticamente la salida de la MLP de la capa interceptada.

---

### Paso 4️: Visualizar la activación interceptada

Revisamos qué se ha capturado y el tamaño de la salida.

```python
print(f"Salida interceptada en la capa {n}:")
print(activaciones_mlp[f'capa_{n}'])
print(f"Forma de la salida: {activaciones_mlp[f'capa_{n}'].shape}")
```
**Explicación de la salida:**
- La salida es un tensor tridimensional de forma `(1, secuencia, dimensiones)`.
- En este caso: `torch.Size([1, 11, 2048])` significa:
  - 1: solo un ejemplo procesado.
  - 11: tokens que forman la secuencia.
  - 2048: dimensión de la representación interna que produce la MLP.
- Cada fila representa las activaciones (características) para un token en la capa interceptada.

---

### Paso 5: Eliminar el hook

Eliminamos el hook para liberar recursos y evitar que siga interceptando salidas.

```python
handle.remove()
**Explicación:**
- El hook consume recursos y permanece activo mientras no se elimine.
- Siempre es buena práctica eliminarlo después de haber capturado la salida deseada.




## Verificando que la extracción de la salida MLP fue exitosa

Después de interceptar la salida de la MLP y ejecutar una inferencia, es importante comprobar que la captura fue realizada correctamente.

#### 1️. Revisar las claves del diccionario
El diccionario `activaciones_mlp` debe contener una clave con el nombre `capa_n`.

```python
print(activaciones_mlp.keys())
```
Salida esperada:
```python
dict_keys(['capa_5'])  # Si n=5
```

#### 2️. Comprobar el tamaño de la activación capturada

```python
print(activaciones_mlp[f'capa_{n}'].shape)
```
La forma típica será:
- `(batch_size, sequence_length, hidden_size)`
- Por ejemplo: `(1, 15, 2048)` si el batch size es 1, el prompt tiene 15 tokens y el `hidden_size` es 2048.

#### 3️. Visualizar valores ejemplo

```python
print(activaciones_mlp[f'capa_{n}'][0, 0, :10])  # Primer token, primeras 10 activaciones
```

---

## Explicación matemática y análisis de la salida de la MLP

Dentro de cada bloque Transformer, la MLP es un mapeo no lineal que transforma la representación obtenida después de la atención.

### Formula general de la MLP

Si la entrada a la MLP es un vector $x \in \mathbb{R}^d$, la MLP realiza la siguiente operación:

$$
\text{MLP}(x) = W_2(\sigma(W_1 x + b_1)) + b_2
$$

Donde:
- $W_1 \in \mathbb{R}^{d_{inter} \times d}$ es la matriz de pesos de la primera capa lineal (expansión).
- $b_1$ es el vector de sesgo (en Llama 3.2 puede ser nulo).
- $\sigma$ es la función de activación no lineal (SiLU).
- $W_2 \in \mathbb{R}^{d \times d_{inter}}$ es la matriz de compresión.
- $b_2$ es el vector de sesgo final.
- $d$ es la dimensión de entrada/salida (ejemplo: 2048).
- $d_{inter}$ es la dimensión intermedia expandida (ejemplo: 8192).

### Desglose paso a paso:

####  Expansión lineal:
$$
z = W_1 x + b_1
$$
Se expande la representación de $d$ a $d_{inter}$.

####  Activación SiLU:
$$
z_{act} = \text{SiLU}(z) = z \cdot \text{sigmoid}(z)
$$
La activación SiLU introduce no linealidad suave, favoreciendo la estabilidad en modelos grandes.

####  Compresión lineal:
$$
y = W_2 z_{act} + b_2
$$
Se reduce de nuevo la representación a la dimensión original $d$.

### Correspondencia con el código del modelo
En el bloque `LlamaMLP`, estas operaciones corresponden a:
- `gate_proj` y `up_proj`: expansión.
- `act_fn = SiLU()`: activación.
- `down_proj`: compresión.

---

## Análisis de las activaciones capturadas

Una vez que interceptamos la salida de la MLP, obtenemos un tensor de forma:
$$
(\text{batch size}, \text{sequence length}, d)
$$

Interpretación:
- La dimensión $d$ es la dimensión de activación (ejemplo: 2048).
- Cada posición de la secuencia tiene un vector de activaciones, representando la respuesta del modelo para ese token en esa capa.

## Visualización básica de las activaciones

### Mostrar el tamaño de las activaciones
```python
# Ver forma de las activaciones
print(activaciones_mlp[f'capa_{n}'].shape)
```

### Graficar histograma de valores
```python
import matplotlib.pyplot as plt

activaciones = activaciones_mlp[f'capa_{n}'].flatten().numpy()

plt.hist(activaciones, bins=100, density=True)
plt.title(f"Distribución de activaciones de la capa {n}")
plt.xlabel("Valor de activación")
plt.ylabel("Frecuencia relativa")
plt.show()
```

### Calcular media y varianza de las activaciones
```python
print("Media de las activaciones:", activaciones.mean())
print("Varianza de las activaciones:", activaciones.var())
```

## Interpretación de la distribución
- Una media cercana a 0 y varianza controlada indican una activación normalizada y estable.
- Valores extremos en la cola del histograma representan "picos" de activación.
- Estos picos suelen corresponder a dimensiones que el modelo usa para tomar decisiones clave.

En la siguiente sección, se aplicará reducción de dimensionalidad (PCA o t-SNE) para visualizar estas representaciones en un espacio 2D o 3D y analizar agrupamientos semánticos.


## Visualización avanzada de las activaciones de la MLP

### Reducción de dimensionalidad con PCA

Las activaciones que hemos capturado son vectores en un espacio de alta dimensión ($d=2048$). Para poder interpretarlas visualmente, usaremos **Análisis de Componentes Principales (PCA)**, que permite proyectar estos datos a un espacio de menor dimensión (usualmente 2D o 3D), preservando la mayor cantidad posible de varianza.

---

###  Explicación matemática de PCA (breve)

Dado un conjunto de datos $X$ con $m$ muestras y $d$ dimensiones, PCA:
1. Centra los datos restando la media.
2. Calcula la matriz de covarianza $C = X^T X / m$.
3. Obtiene los vectores propios y valores propios de $C$.
4. Ordena esos vectores propios y selecciona los primeros $k$ para proyectar los datos en un subespacio de dimensión $k$.

---

### Paso 1️: Preparar los datos para PCA

```python
import numpy as np
from sklearn.decomposition import PCA

# Extraemos el tensor de activaciones (ejemplo: capa n)
activaciones_tensor = activaciones_mlp[f'capa_{n}']  # Shape: (batch_size, seq_length, hidden_dim)

# Convertimos el tensor en un arreglo 2D: (batch_size * seq_length, hidden_dim)
activaciones_2d = activaciones_tensor.reshape(-1, activaciones_tensor.shape[-1]).numpy()
```

---

### Paso 2️: Aplicar PCA

```python
# Aplicamos PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
activaciones_pca = pca.fit_transform(activaciones_2d)

# Información sobre varianza explicada
print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)
```

**Salida esperada:**
- Un array con dos valores que indican qué porcentaje de la varianza original es explicada por cada componente.
- Ejemplo:
```text
Varianza explicada por cada componente: [0.35 0.20]
```
Esto significaría que el primer componente retiene el 35% de la varianza y el segundo el 20%.

---

### Paso 3️: Graficar las activaciones proyectadas

```python
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(activaciones_pca[:, 0], activaciones_pca[:, 1], alpha=0.3, s=10)
plt.title(f"Proyección PCA de las activaciones de la capa {n}")
plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.grid(True)
plt.show()
```

**Salida esperada:**
- Un gráfico de dispersión (scatter plot) donde cada punto representa una posición de token proyectada a 2D.
- Pueden aparecer nubes densas o grupos de puntos, lo que indica que las activaciones tienen patrones estructurados.

---

En la siguiente sección explicaremos cómo interpretar estas agrupaciones y qué significan en el contexto de la interpretabilidad mecanicista de transformers.


## Interpretación de las agrupaciones observadas en PCA

Una vez que hemos proyectado las activaciones en 2D, podemos analizar visualmente los patrones:

### ¿Qué representan los grupos o nubes de puntos?
- Cada punto en el gráfico corresponde a un vector de activación para un token específico dentro del lote de entrada.
- Grupos densos indican que hay tokens o posiciones que comparten representaciones internas similares.
- Puntos alejados o dispersos pueden representar outliers o tokens con roles especiales (por ejemplo, delimitadores o símbolos que la red trata de manera particular).

---

### Ejemplos de interpretación:
- Si observamos una nube central muy densa con algunas ramas o salidas laterales, esto indica que la mayoría de las activaciones están cerca de un espacio común, pero ciertos tokens activan dimensiones específicas.
- Si aparecen subgrupos claramente diferenciados, puede ser evidencia de clústers semánticos: grupos de tokens con funciones similares.

---

## Visualización avanzada con t-SNE 

El algoritmo **t-SNE** (t-distributed Stochastic Neighbor Embedding) es una técnica de reducción de dimensionalidad no lineal, ideal para visualizar agrupaciones y relaciones locales.

### Paso 1️: Aplicar t-SNE

```python
from sklearn.manifold import TSNE

# Reducimos primero con PCA a 50 dimensiones para mayor eficiencia
pca_50 = PCA(n_components=50)
activaciones_pca50 = pca_50.fit_transform(activaciones_2d)

# Ahora aplicamos t-SNE
print("Ejecutando t-SNE... (puede tardar unos segundos)")
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=1000, verbose=1)
activaciones_tsne = tsne.fit_transform(activaciones_pca50)
```

### Paso 2️: Graficar la proyección t-SNE

```python
plt.figure(figsize=(8, 6))
plt.scatter(activaciones_tsne[:, 0], activaciones_tsne[:, 1], alpha=0.4, s=10, cmap="viridis")
plt.title(f"Visualización t-SNE de las activaciones de la capa {n}")
plt.xlabel("Dimensión 1")
plt.ylabel("Dimensión 2")
plt.grid(True)
plt.show()
```

---

## Interpretación de t-SNE
- t-SNE permite observar relaciones locales: puntos cercanos en el gráfico representan activaciones similares.
- Clusters bien definidos son evidencia de estructuras internas aprendidas por la red.
- Si observamos brazos o ramificaciones, pueden indicar caminos de transformación de información desde representaciones generales a específicas.




## Comparando activaciones de la MLP entre diferentes prompts

Para entender aún más la interpretabilidad mecanicista, es útil comparar cómo cambian las activaciones de la MLP al variar el texto de entrada.

---

### Paso 1️: Definir diferentes prompts

```python
prompts = [
    "La inteligencia artificial está transformando la educación.",
    "La biología estudia los seres vivos y su entorno.",
    "Las matemáticas son el lenguaje del universo.",
    "El arte es una forma de expresión humana."
]
```

---

### Paso 2️: Capturar las activaciones para cada prompt

```python
resultados_prompts = {}

for idx, prompt in enumerate(prompts):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Interceptar con hook
    handle = model.model.layers[n].mlp.register_forward_hook(guardar_salida_mlp(f"prompt_{idx}"))
    
    with torch.no_grad():
        _ = model(**inputs)
    
    resultados_prompts[f'prompt_{idx}'] = activaciones_mlp[f'prompt_{idx}']
    handle.remove()
```

---

### Paso 3️: Visualizar comparando prompts

Podemos graficar histogramas superpuestos:

```python
plt.figure(figsize=(10, 6))

for idx, prompt in enumerate(prompts):
    activaciones = resultados_prompts[f'prompt_{idx}'].flatten().numpy()
    plt.hist(activaciones, bins=100, alpha=0.3, density=True, label=f'Prompt {idx+1}')

plt.title(f"Comparación de distribuciones de activaciones en la capa {n}")
plt.xlabel("Valor de activación")
plt.ylabel("Frecuencia relativa")
plt.legend()
plt.show()
```

---

## Interpretación
- Si las distribuciones son similares, la capa MLP está respondiendo de forma general.
- Diferencias notables entre prompts indican especialización: ciertas dimensiones se activan más según el tipo de entrada.
- Esto respalda la idea de que la MLP refina información contextual.


## Conclusiones y recomendaciones

### Conclusiones principales:
1. **La MLP en cada bloque Transformer** actúa como un refinador de representaciones, expandiendo y comprimiendo la información después de la atención.
2. **Las activaciones de la MLP** son vectores en espacios de alta dimensión que muestran estructuras internas, evidenciadas por patrones y agrupaciones cuando se proyectan a 2D.
3. **La variación en las activaciones entre diferentes prompts** revela que la red ajusta sus representaciones dependiendo del contenido semántico de la entrada.
4. Las herramientas de reducción de dimensionalidad como **PCA** y **t-SNE** permiten visualizar y entender mejor la estructura interna de las activaciones.

---

###  Recomendaciones para futuros análisis:
- Probar con más prompts y dominios temáticos (ciencia, arte, política) para observar especialización.
- Realizar análisis estadísticos sobre las activaciones para medir dispersión, concentración y valores extremos.
- Comparar activaciones entre capas tempranas y capas tardías para ver cómo evoluciona la información.
- Investigar qué dimensiones de la MLP dominan las salidas y cómo están correlacionadas con tokens clave.

---

###  Resumen para anexar al reporte:
- Se explicó formalmente qué es la MLP y su función dentro del modelo Llama 3.2.
- Se mostró cómo interceptar la salida de la MLP utilizando hooks.
- Se analizaron las activaciones obtenidas mediante visualización básica y avanzada.
- Se compararon las respuestas del modelo ante diferentes prompts.
- Se plantearon recomendaciones para un análisis más profundo en proyectos de interpretabilidad.

Este análisis forma una base sólida para demostrar comprensión teórica, capacidad práctica y análisis crítico en el proyecto de interpretabilidad mecanicista de transformers.
